## 开源大模型结合外部知识库的自动问答

开源大语言模型有着部署成本低，输出可控等优点。不过对于一些较小参数的模型，例如 Llama2-7B，Zephyr-7B 来说，在回答一些很细节的问题时经常会出现幻觉。这些幻觉会影响模型最终输出的准确性。因此，我们需要将外部知识库引入到生成过程中，提高生成内容的准确度和可信度。

### 安装依赖

In [ ]:
%pip install langchain langchain-experimental text_generation InstructorEmbedding replicate --upgrade

In [ ]:
# %pip install getpass
%pip install sqlalchemy==1.4.48
%pip install clickhouse-sqlalchemy==0.2.4

In [71]:
from sqlalchemy import __version__
print(__version__)

1.4.48


### 使用 Bedrock 的 Claude 模型和推理资源

In [72]:
import boto3
session = boto3.Session()
bedrock_client = session.client(
    service_name='bedrock-runtime',
    region_name='us-east-1'
)

In [73]:
from langchain.llms.bedrock import Bedrock

inference_modifier = {
    "max_tokens_to_sample": 4096,
    "temperature": 0.5,
    "top_k": 250,
    "top_p": 1,
    "stop_sequences": ["\n\nHuman"],
}

claude_llm = Bedrock(
    model_id="anthropic.claude-v2",
    client=bedrock_client,
    model_kwargs=inference_modifier,
)
prompt2 = "When did Geoffrey Hinton born?"
claude_llm(prompt2)

' Geoffrey Hinton was born on December 6, 1947 in Wimbledon, England. He is a British-Canadian cognitive psychologist and computer scientist. Hinton is a leading figure in the field of artificial neural networks and machine learning.'

通过维基百科我们可以知道：
<iframe
	src="https://en.wikipedia.org/wiki/Geoffrey_Hinton"
	frameborder="0"
	width="1080"
	height="500"
></iframe>

Geoffrey Hinton 的生日并不是 3 月 12 日，因此我们需要外部知识的帮助。

### 构建搜索：创建 Embedding 模型

In [74]:
from langchain.embeddings import SentenceTransformerEmbeddings

emb_model = SentenceTransformerEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
)


### 构建搜索：连接数据库

In [75]:
from sqlalchemy import create_engine, MetaData

MYSCALE_USER = "chatdata"
MYSCALE_PASSWORD = "myscale_rocks"
MYSCALE_HOST = "msc-1decbcc9.us-east-1.aws.staging.myscale.cloud"
MYSCALE_PORT = 443

engine = create_engine(
    f"clickhouse://{MYSCALE_USER}:{MYSCALE_PASSWORD}@{MYSCALE_HOST}:{MYSCALE_PORT}/wiki?protocol=https"
)
metadata = MetaData(bind=engine)


### 构建搜索：构建查询构造器

#### 关于 `Vector SQL`

<img src="https://myscale.com/blog/assets/img/pipeline.015b6008.png" height="300px">

由于带有向量搜索的 SQL 与常规 SQL 非常相似，我们可以让大语言模型来生成一个向量搜索的中间形式：也就是 `Vector SQL`

```sql
SELECT * FROM table
ORDER BY DISTANCE(vector, NeuralArray(flower))
LIMIT 10
```

通过 prompt 我们可以让语言模型学会使用距离函数 `DISTANCE` 和 文本特征提取函数 `NeuralArray`
与此同时还可以让它学会随意组合不同的过滤条件。这样就可以更加自动地构建用户所期望的搜索查询了。

下面是我们用来将语言模型输出转化为向量搜索 SQL 的代码：


In [76]:
from typing import List, Dict, Any
from langchain_experimental.sql.vector_sql import VectorSQLOutputParser


class VectorSQLRetrieveCustomOutputParser(VectorSQLOutputParser):
    """Based on VectorSQLOutputParser
    It also modify the SQL to get all columns
    """

    must_have_columns: List[str]

    @property
    def _type(self) -> str:
        return "vector_sql_retrieve_custom"

    def parse(self, text: str) -> Dict[str, Any]:
        start = text.upper().find("SELECT")
        if start >= 0:
            text = text[start:-1]
            end = text.upper().find("FROM")
            text = text.replace(
                text[start + len("SELECT") + 1: end - 1],
                ", ".join(self.must_have_columns),
                1
            )
        qstr = super().parse(text)
        return qstr


### 构建搜索：集成 LLM 与 数据库

In [120]:
import prompts_claude
import importlib

importlib.reload(prompts_claude)

<module 'prompts_claude' from '/home/ec2-user/SageMaker/workshops/myscale-aws-workshop/chat_with_database/prompts_claude.py'>

In [121]:
from prompts_claude import _myscale_prompt
from langchain.prompts import PromptTemplate
from langchain.sql_database import SQLDatabase
from langchain_experimental.retrievers.vector_sql_database import (
    VectorSQLDatabaseChainRetriever,
)
from langchain_experimental.sql.vector_sql import VectorSQLDatabaseChain
from langchain.llms import HuggingFaceTextGenInference


must_have_cols = ['id', 'title', 'url', 'text', 'views']

PROMPT = PromptTemplate(
    input_variables=["input", "table_info", "top_k"],
    template=_myscale_prompt,
)
output_parser = VectorSQLRetrieveCustomOutputParser.from_embeddings(
    model=emb_model, must_have_columns=must_have_cols
)

sql_query_chain = VectorSQLDatabaseChain.from_llm(
    llm=claude_llm,
    prompt=PROMPT,
    top_k=10,
    return_direct=True,
    db=SQLDatabase(engine, None, metadata, max_string_length=1024),
    sql_cmd_parser=output_parser,
    native_format=True,
)
sql_retriever = VectorSQLDatabaseChainRetriever(
    sql_db_chain=sql_query_chain, page_content_key="text"
)


### 执行查询

In [122]:
from langchain.callbacks import StdOutCallbackHandler

docs = sql_retriever.get_relevant_documents("When did Geoffrey Hinton born?",
                                            callbacks=[StdOutCallbackHandler()])
docs




> Entering new VectorSQLDatabaseChain chain...
When did Geoffrey Hinton born?
SQLQuery:

> Entering new LLMChain chain...
Prompt after formatting:
You are a MyScale expert. You are provided with a database table schema and several sample results of the table. Use MyScale syntax to generate a sql query for the input question.
These are some rules for MyScale syntax.
 # MyScale queries has a vector distance function called `DISTANCE(column, array)` to compute relevance to the user's question and sort the feature array column by the relevance. 
 # When the query is asking for 10 closest row, you have to use this distance function to calculate distance to entity's array on vector column and order by the distance to retrieve relevant rows.
 # *NOTICE*: `DISTANCE(column, array)` only accept an array column as its first argument and a `NeuralArray(entity)` as its second argument. You also need a user defined function called `NeuralArray(entity)` to retrieve the entity's array. 
 # Unless th

DatabaseException: Orig exception: Code: 62. DB::Exception: Syntax error: failed at position 1 ('<') (line 1, col 1): <table_info>
CREATE TABLE "Wikipedia" (
	id String, 
	title String, 
	text String, 
	url String, 
	wiki_id UInt64, 
	views Float32, 
	paragraph_id UInt64, 
	lan. Expected one of: Query, Query with output, EXPLAIN, SELECT query, possibly with UNION, list of union elements, SELECT query, subquery, possibly with UNION, SELECT subquery, SELECT query, WITH, FROM, SELECT, SHOW CREATE QUOTA query, SHOW CREATE, SHOW [FULL] [TEMPORARY] TABLES|DATABASES|CLUSTERS|CLUSTER 'name' [[NOT] [I]LIKE 'str'] [LIMIT expr], SHOW, SHOW ENGINES query, SHOW ENGINES, EXISTS or SHOW CREATE query, EXISTS, DESCRIBE FILESYSTEM CACHE query, DESCRIBE, DESC, DESCRIBE query, SHOW PROCESSLIST query, SHOW PROCESSLIST, CREATE TABLE or ATTACH TABLE query, CREATE, ATTACH, REPLACE, CREATE DATABASE query, CREATE VIEW query, CREATE DICTIONARY, CREATE LIVE VIEW query, CREATE WINDOW VIEW query, ALTER query, ALTER

In [123]:
docs = sql_retriever.get_relevant_documents("What books did Geoffrey Hinton write?",
                                            callbacks=[StdOutCallbackHandler()])
docs




> Entering new VectorSQLDatabaseChain chain...
What books did Geoffrey Hinton write?
SQLQuery:

> Entering new LLMChain chain...
Prompt after formatting:
You are a MyScale expert. You are provided with a database table schema and several sample results of the table. Use MyScale syntax to generate a sql query for the input question.
These are some rules for MyScale syntax.
 # MyScale queries has a vector distance function called `DISTANCE(column, array)` to compute relevance to the user's question and sort the feature array column by the relevance. 
 # When the query is asking for 10 closest row, you have to use this distance function to calculate distance to entity's array on vector column and order by the distance to retrieve relevant rows.
 # *NOTICE*: `DISTANCE(column, array)` only accept an array column as its first argument and a `NeuralArray(entity)` as its second argument. You also need a user defined function called `NeuralArray(entity)` to retrieve the entity's array. 
 # Un

DatabaseException: Orig exception: Code: 62. DB::Exception: Syntax error: failed at position 1 ('<') (line 1, col 1): <table_info>

CREATE TABLE "Wikipedia" (
	id String, 
	title String, 
	text String, 
	url String, 
	wiki_id UInt64, 
	views Float32, 
	paragraph_id UInt64, 
	la. Expected one of: Query, Query with output, EXPLAIN, SELECT query, possibly with UNION, list of union elements, SELECT query, subquery, possibly with UNION, SELECT subquery, SELECT query, WITH, FROM, SELECT, SHOW CREATE QUOTA query, SHOW CREATE, SHOW [FULL] [TEMPORARY] TABLES|DATABASES|CLUSTERS|CLUSTER 'name' [[NOT] [I]LIKE 'str'] [LIMIT expr], SHOW, SHOW ENGINES query, SHOW ENGINES, EXISTS or SHOW CREATE query, EXISTS, DESCRIBE FILESYSTEM CACHE query, DESCRIBE, DESC, DESCRIBE query, SHOW PROCESSLIST query, SHOW PROCESSLIST, CREATE TABLE or ATTACH TABLE query, CREATE, ATTACH, REPLACE, CREATE DATABASE query, CREATE VIEW query, CREATE DICTIONARY, CREATE LIVE VIEW query, CREATE WINDOW VIEW query, ALTER query, ALTER

### 构建 RAG：将外部知识连接至生成提示中

首先，我们使用了之前构造好的 VectorSQL 检索器。同时我们使用提示模板将他们整理好嵌入进生成提示中。

我们这里使用了 LangChain 的 `RetrievalQAwithSources` 提示链。

In [59]:
from langchain import LLMChain
from langchain.chains.qa_with_sources.retrieval import RetrievalQAWithSourcesChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

combine_prompt_template = (
    "You are a helpful document assistant. Your task is to answer any questions "
    + "related to the given documents. You should use the title and abstract of the selected documents as your source of information "
    + "and try to provide concise and accurate answers to any questions asked by the user. If you are unable to find "
    + "relevant information in the given sections, you will need to let the user know that the source does not contain "
    + "relevant information but still try to provide an answer based on your general knowledge. The following is the related information "
    + "about the document that will help you answer users' questions.\nHere the contexts:\n{summaries}\n\n\nQuestion: {question}"
    + "\nAnswer: "
)

COMBINE_PROMPT = PromptTemplate(
    input_variables=["summaries", "question"], template=combine_prompt_template)

doc_prompt = PromptTemplate(
            input_variables=["page_content", "url", "title"],
            template="Title: {title}\nContent: {page_content}\nSOURCE: {url}")

chain = RetrievalQAWithSourcesChain(
    retriever=sql_retriever,
    combine_documents_chain=StuffDocumentsChain(
        llm_chain=LLMChain(
            prompt=COMBINE_PROMPT,
            llm=claude_llm,
        ),
        document_prompt=doc_prompt,
        document_variable_name="summaries",

    ),
    return_source_documents=True,
    max_tokens_limit=12000,
)


In [60]:
chain("When did Geoffrey Hinton born?", callbacks=[StdOutCallbackHandler()])



> Entering new RetrievalQAWithSourcesChain chain...


> Entering new VectorSQLDatabaseChain chain...
When did Geoffrey Hinton born?
SQLQuery:

> Entering new LLMChain chain...
Prompt after formatting:
Human: You are a MyScale expert. You are provided with a database table schema and several sample results of the table. Write a SQL query according to the input question with the following rules.
Given an input question, first create a syntactically correct MyScale query to run, then look at the results of the query and return the answer to the input question.
MyScale queries has a vector distance function called `DISTANCE(column, array)` to compute relevance to the user's question and sort the feature array column by the relevance. 
When the query is asking for 10 closest row, you have to use this distance function to calculate distance to entity's array on vector column and order by the distance to retrieve relevant rows.
*NOTICE*: `DISTANCE(column, array)` only accept an array column

{'question': 'When did Geoffrey Hinton born?',
 'answer': ' Unfortunately the given documents do not provide information about when Geoffrey Hinton was born. However, based on my general knowledge, Geoffrey Hinton is a computer scientist and cognitive psychologist known for his work on artificial neural networks. He was born in 1947 in the United Kingdom. So the answer is that Geoffrey Hinton was born in 1947.',
 'sources': '',
 'source_documents': []}